<a href="https://colab.research.google.com/github/jrgosalvez/data255_DL/blob/main/HW12-Chatbot/Jorge_Gosalvez_DL255_HW12_rag_chatbot_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SJSU MSDS 255 DL, Spring 2024 - Building RAG Chatbots with LangChain
Homework 12 - Part B: Custom Data Chatbot

Git: https://github.com/jrgosalvez/data255_DL

Sources:
* SJSU DL 255 RAG Chatbot with LangChain demo
* [OpenAI API key](https://platform.openai.com/account/api-keys) and [Pinecone API key](https://app.pinecone.io)
* [OpenAI Embeddings](https://platform.openai.com/docs/guides/embeddings/use-cases)
* [RAG on Complex PDFs with Langchain](https://medium.com/the-ai-forum/rag-on-complex-pdf-using-llamaparse-langchain-and-groq-5b132bd1f9f3)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb)

### Part B Goal

Build a chatbot to answer questions based on custom data from multiple documents using LangChain, OpenAI, and Pinecone vector DB, to build a chatbot capable of learning from the external world using **R**etrieval **A**ugmented **G**eneration (RAG).

The chatbot will save the conversation in memory such that it can expand on the conversation based on the past and summarize the conversation.

### Prerequisites

Install the following Python libraries:

- **langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **datasets**: This library provides a vast array of datasets for machine learning. We'll use it to load our knowledge base for the chatbot.
- **pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone API and store our chatbot's knowledge base in a vector database.

**NOTE**: *OpenAI dataloaders will not load locally for on-prem devices easily. To simplify the use of these loaders, it is recommended to use an online notebook such as CoLab.*

In [1]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    datasets==2.10.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### BACKGROUND: Building a Chatbot (no RAG)

We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a `ChatOpenAI` object. For this we do need an [OpenAI API key](https://platform.openai.com/account/api-keys).

In [2]:
import os
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to know when Hollow Knight Silksong is releasing.")
]

The format is very similar, we're just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [4]:
res = chat(messages)
res

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="As of now, Team Cherry has not announced an official release date for Hollow Knight: Silksong. The game is currently in development, and fans are eagerly awaiting news on its release. I recommend keeping an eye on Team Cherry's official announcements for updates on the game's release date.")

In response we get another AI message object. We can print it more clearly like so:

In [5]:
print(res.content)

As of now, Team Cherry has not announced an official release date for Hollow Knight: Silksong. The game is currently in development, and fans are eagerly awaiting news on its release. I recommend keeping an eye on Team Cherry's official announcements for updates on the game's release date.


### Stringing Messages for a Conversation
Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do Hollow Knight fans want Silksong?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Hollow Knight: Silksong is highly anticipated by fans for several reasons. The original Hollow Knight game received critical acclaim for its beautiful hand-drawn art style, challenging gameplay, deep lore, and atmospheric world. Fans of the original game are excited to explore a new world in Silksong, with new characters, enemies, abilities, and challenges to discover. The protagonist of Silksong, Hornet, was a fan-favorite character from the first game, and players are eager to learn more about her backstory and see how her adventure unfolds in the new game. Overall, fans of Hollow Knight are excited for Silksong because they trust Team Cherry to deliver another captivating and immersive experience in the same vein as the original game.


### Dealing with Hallucinations

We have our chatbot, but as mentioned — the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about the new (and very popular) Llama 2 LLM.

In [7]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Silksong?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [8]:
print(res.content)

Hollow Knight: Silksong is special for several reasons. Here are some key aspects that make it stand out:

1. **New Protagonist**: Silksong features Hornet as the playable character, offering players a fresh perspective and a new storyline to explore.

2. **New World**: Silksong introduces players to a brand new kingdom with unique environments, characters, and challenges to discover.

3. **Expanded Gameplay**: Silksong promises new abilities, mechanics, and enemies, providing players with fresh gameplay experiences and challenges.

4. **Enhanced Visuals**: Building upon the beautiful hand-drawn art style of the original game, Silksong offers stunning visuals and animations that bring the world to life.

5. **Deep Lore**: Like its predecessor, Silksong is expected to have a rich and mysterious lore for players to uncover, adding depth and intrigue to the game's world.

6. **Challenging Gameplay**: Silksong is known for its challenging gameplay that rewards skill and perseverance, offer

Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

OpenAI have since adjusted the behavior for this particular example as we can see below:

In [9]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me when Silksong was released?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [10]:
print(res.content)

As of my last update, Hollow Knight: Silksong has not been released yet. The game is still in development, and an official release date has not been announced by Team Cherry. Fans are eagerly awaiting news on the game's release, so I recommend keeping an eye on Team Cherry's official announcements for updates on the release date.


### Importing the Data

In [11]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.9 MB/s eta 0:00:00


In [13]:
from langchain.document_loaders import PyPDFDirectoryLoader

#load pdf files
loader = PyPDFDirectoryLoader('/content/drive/MyDrive/MSDA/DATA255/silksongPDF')
data = loader.load()
print(data)

[Document(page_content='Hollow Knight: Silksong\nD e v e l o p e r ( s ) Team Cherry\nP u b l i s h e r ( s ) Team Cherry\nD e s i g n e r ( s ) Ari Gibson\nWilliam Pellen\nP r o g r a m m e r ( s )William Pellen\nJack Vine\nA r t i s t ( s ) Ari Gibson\nC o m p o s e r ( s ) Christopher\nLarkin\nE n g i n e Unity\nP l a t f o r m ( s ) Linux\nmacOS\nMicrosoft\nWindows\nNintendo Switch\nPlayStation 4\nPlayStation 5\nXbox One\nXbox Series X/S\nM o d e ( s ) Single-player\nHollow Knight: Silksong\nHollow Knight: Silksong  is an upcoming Metroidvania  video\ngame  in develo pment by Australian independent developer  Team\nCherry. The sequel to Hollow Knight , it is being developed for\nWindows , macOS , Linux , PlayStation 4, PlayStation 5, Nintendo\nSwitch , Xbox One  and Xbox Series X/S .\nSilksong  was originally conceived as downloadable content  for\nHollow Knight , but the scope of the project grew enough that on\nFebruary 14, 2019, Team Cherry announced it as a separate\nsequel.[1]

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split text data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)
print(len(text_chunks))

36


In [15]:
# check the chunks
text_chunks[2]

Document(page_content='sporadic and infrequent".[9]Gameplay\nPremise\nDevelopment5/5/24, 3:39 PM Hollow Knight: Silksong - Wikipedia\nhttps://en.wikipedia.org/wiki/Hollow_Knight:_Silksong 1/4', metadata={'source': '/content/drive/MyDrive/MSDA/DATA255/silksongPDF/HollowKnightSilksongWikipedia.pdf', 'page': 0})

In [16]:
# reformat chunks to improve vectorization; match 'jamescalam/llama-2-arxiv-papers-chunked' format sourced from Llama 2 ArXiv papers on huggingface
dataset = []

for i, chunk in enumerate(text_chunks):
    dataset.append({
        'doi': '',  # you can add a DOI here if available
        'chunk-id': str(i),
        'chunk': chunk,
        'id': '',  # you can add an ID here if available
        'title': '',  # you can add a title here if available
        'summary': '',  # you can add a summary here if available
        'source': '',  # you can add a source here if available
        'authors': [],  # you can add authors here if available
        'categories': [],  # you can add categories here if available
        'comment': '',  # you can add a comment here if available
        'journal_ref': None,  # you can add a journal reference here if available
        'primary_category': '',  # you can add a primary category here if available
        'published': '',  # you can add a published date here if available
        'updated': '',  # you can add an updated date here if available
        'references': []  # you can add references here if available
    })

print(dataset[3])

{'doi': '', 'chunk-id': '3', 'chunk': Document(page_content='Eventually, due to the increased scope of the project, Team Cherry decided to expand the\ndownloadable content to a full sequel.[1][2]\nThe developers released an update in March 2019, sharing descriptions and images of characters who\nwill appe ar in Silksong . They thanked the game\'s fans for supporting them regarding the\nannouncement of the sequel.[10]\nIn Decem ber 2019 , Team Cherry released a preview of the soundtrack, composed by Christopher\nLarkin , as well as an update on the total number of enemies developed, with a focus on a trio,\ndescribed as "members of a scholarly suite."[11]\nAn article in PC Gamer  from February 2022 showcased Team Cherry co-director William Pellen\nstating that the game was still in development despite the lack of updates since December 2019 and\nsaid that more details would be revealed as the game got closer to its release.[12]\nA new trailer was revealed at the Xbox & Bethesda Games Sh

#### Dataset Overview

The dataset used are PDFs samples of my (Silksong Gosalvez's) Deep Learning homeworks.

Because most **L**arge **L**anguage **M**odels (LLMs) only contain knowledge of the world as it was during training, they cannot answer our questions about Silksong the game without example data.

### Task 4: Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our connection to Pinecone, this requires a [free API key](https://app.pinecone.io).

In [17]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key= userdata.get('PineCone')

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [18]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Then we initialize the index. We will be using OpenAI's `text-embedding-ada-002` model for creating the embeddings, so we set the `dimension` to `1536`.

In [19]:
import time

index_name = 'llama-2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Our index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings we will OpenAI's `text-embedding-ada-002` model — we can access it via LangChain like so:

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Using this model we can create embeddings like so:

In [21]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

From this we get two (aligning to our two chunks of text) 1536-dimensional embeddings.

We're now ready to embed and index all our our data! We do this by looping through our dataset and embedding and inserting everything in batches.

**NOTE**: *ensure that chunks are strings and ensure that they are correctly assigned to metadata (do this with the .page_content method)*

In [22]:
import pandas as pd
from tqdm.auto import tqdm  # for progress bar

data = pd.DataFrame(dataset) # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [str(x['chunk']) for _, x in batch.iterrows()]

    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'].page_content,
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/1 [00:00<?, ?it/s]

We can check that the vector index has been populated using `describe_index_stats` like before:

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 36}},
 'total_vector_count': 36}

#### Retrieval Augmented Generation

We've built a fully-fledged knowledge base. Now it's time to connect that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.

In [24]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Silksong's prior deep learning homeworks.

In [25]:
query = "Did Silksong get released?"

vectorstore.similarity_search(query, k=3)

[Document(page_content='Silksong  would be released on Xbox Game Pass  at launch, with the game being available through the\nservice for PC and Xbox Series X/S.[13] While no release date was announ ced, the Xbox Twitter\naccount stated in a tweet that it would be available within the next twelve months, implying they\nexpected a release by 12 June 2023.[14][15] Team Cherry marketing and publishing representative\nMatthew Griffin declared on 10 May 2023 that the game was delayed, stating "We had planned to\nrelease in the first half of 2023, but development is still continuing."[16]\nIn September 2022, Sony  confirmed in a tweet that the game  would also come to PlayStation 4 and\nPlayStation 5.[17]\nIn May 2022, Hollow Knight: Silksong  won a "Most Anticipated Game" award from Unity . In\nresponse, Team Cherry thanked the community for their support and said "It can\'t be too much\nlonger, surely!"[9][18][19]\n1. "Team Cherry Holiday Sign of f" (https://www .teamcherry .com.au/blog/1 1

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to connect the output from our `vectorstore` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [26]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [27]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Silksong  would be released on Xbox Game Pass  at launch, with the game being available through the
service for PC and Xbox Series X/S.[13] While no release date was announ ced, the Xbox Twitter
account stated in a tweet that it would be available within the next twelve months, implying they
expected a release by 12 June 2023.[14][15] Team Cherry marketing and publishing representative
Matthew Griffin declared on 10 May 2023 that the game was delayed, stating "We had planned to
release in the first half of 2023, but development is still continuing."[16]
In September 2022, Sony  confirmed in a tweet that the game  would also come to PlayStation 4 and
PlayStation 5.[17]
In May 2022, Hollow Knight: Silksong  won a "Most Anticipated Game" award from Unity . In
response, Team Cherry thanked the community for their support and said "It can't be too much
longer, surely!"[9][18][19]
1. "Team Cherry Holiday Sign of f" (https://www .

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [28]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

As of mid-February 2024, Team Cherry has not shared any further details on the release date for Hollow Knight: Silksong. Therefore, Silksong has not been released yet.


We can continue with more questions about Silksong's prior deep learning homeworks. Let's try _without_ RAG first:

In [29]:
prompt = HumanMessage(
    content="What systems will Silksong release on?"
)

res = chat(messages + [prompt])
print(res.content)

Hollow Knight: Silksong will be released on PS5, Xbox Series X|S, Nintendo Switch, and PC. Additionally, the game will be available on Xbox Game Pass on launch day.


The chatbot is able to respond about Silksong's prior deep learning homeworks thanks to it's conversational history stored in `messages`.

In [30]:
prompt = HumanMessage(
    content=augment_prompt(
        "Who created Silksong?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Hollow Knight: Silksong is being developed by the Australian independent developer Team Cherry.


In [31]:
prompt = HumanMessage(
    content=augment_prompt(
        "What date should fans pay attention to?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Fans of Hollow Knight: Silksong should pay attention to April 29, as there is a potential opportunity for an update on the game's status during an Xbox Digital Showcase on that date. This event could feature new information about the progress towards the release of Silksong and may offer updates that fans have been eagerly anticipating.


In [32]:
prompt = HumanMessage(
    content=augment_prompt(
        "Who wrote about Silksong on april 23, 2024?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Ben Brosofsky wrote about Silksong on April 23, 2024.


In [33]:
prompt = HumanMessage(
    content=augment_prompt(
        "What is the only games confirmed for the new IGN x ID@Xbox Digital Showcase?"
    )
)

res = chat(messages + [prompt])
print(res.content)

The only games confirmed for the new IGN x ID@Xbox Digital Showcase are Dungeons of Hinterberg, 33 Immortals, and Lost Records Bloom & Rage, as mentioned in the Xbox Wire announcement.


In [35]:
prompt = HumanMessage(
    content=augment_prompt(
        "Who said 'Discovery is a huge part of Hollow Knight so we don’t want to spoil all the new systems and surprises'?"
    )
)

res = chat(messages + [prompt])
print(res.content)

The statement "Discovery is a huge part of Hollow Knight so we don’t want to spoil all the new systems and surprises" was made by Team Cherry during their Hollow Knight: Silksong reveal.


In [38]:
prompt = HumanMessage(
    content=augment_prompt(
        "Who is Jordan Sirani?"
    )
)

res = chat(messages + [prompt])
print(res.content)

I'm sorry, but the provided contexts do not mention anyone named Jordan Sirani. If there is any other information or context you can provide, I would be happy to try to assist further.


In [36]:
prompt = HumanMessage(
    content=augment_prompt(
        "What 'mode' will Silksong feature?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Silksong will feature Silk Soul Mode, which will be available after you complete the game for the first time. This mode, similar to Steel Soul Mode in Hollow Knight, is described as spinning the game into a unique and challenging experience. Further details about Silk Soul Mode have not yet been announced.


In [37]:
prompt = HumanMessage(
    content=augment_prompt(
        "Summarize our chat in bullets."
    )
)

res = chat(messages + [prompt])
print(res.content)

- Hollow Knight: Silksong has not been released yet.
- The game was initially expected to be released in the first half of 2023.
- Team Cherry announced a delay in the game's release, citing continued development to ensure the game's quality.
- As of mid-February 2024, there have been no further details shared on the release date.
- Hollow Knight: Silksong is anticipated to be released on PS5, Xbox Series X|S, Nintendo Switch, and PC, with availability on Xbox Game Pass at launch.


**Observations and Limitations:**
* PDFs can include special characters and formatting complexities, so the LLM did not grab all details, for example the RAG grabbed context written by Jordan Sirani, but could not determine that he wrote the article
* chunking format ensures data loading and ingestion occurs properly
* appending prompts and responses to messages expand content to enable the chatbot to 'converse'
* savign messages by passing them forward allows the chatbot to 'remember' the conversation for conclusions and anslysis

Delete the index to save resources and not be charged for non-use:

In [39]:
pc.delete_index(index_name)

---